In [9]:
path=u'/Users/NoraChan/Desktop/python for data/ch2/usagov_bitly_data2012-03-16-1331923249'
#unicode 编码
open(path).readline()

import json
records=[json.loads(line) for line in open(path)] #json模块load函数加载已经下载好的数据文件；turn json into dict//列表推导式
time_zones=[rec['tz'] for rec in records if 'tz'in rec] #type(records):list,type(records[1]):dict,type(time_zones):list

#统计时区using标准库
def get_counts(sequence):
    counts={}
    for x in time_zones:
        if x in counts:
            counts[x]+=1
        else:
            counts[x]=1        
    return counts


#统计时区using标准库
from collections import defaultdict

def get_counts2(sequence):
    counts=defaultdict(int)#所有值初始化为0
    for x in sequence:
        counts[x]+=1
    return counts

#获取前十位时区&计数值using标准库
def top_counts(count_dict,n=10):
    value_key_pairs=[(count,tz) for tz,count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]

#获取前十位时区&计数值using标准库
from collections import Counter
counts=Counter(time_zones)
counts.most_common(10)

#引入pandas，用表格表达数据
from pandas import DataFrame,Series
import pandas as pd;import numpy as np
frame=DataFrame(records)#frame[:2]==frame.ix[:1]#这个判断的值是？

#统计时区using pandas
frame['tz']    #返回Seris类型，返回该列数据
tz_counts=frame['tz'].value_counts()
tz_counts[:10]

#替换NA、空字符串
clean_tz=frame['tz'].fillna('Missing')
clean_tz[clean_tz=='']='Unknown'#type(clean_tz==''):Series

tz_counts=clean_tz.value_counts()
tz_counts[:10]

#利用matplotlib生成水平条形图（先清除NA、空字符串）
tz_counts[:10].plot(kind='barh',rot=0)

#利用字符串函数＋正则表达式解析user_agent
results=Series([x.split()[0] for x in frame.a.dropna()])#Series数据结构,frame.a 代表DataFrame中的a字段


'''
移除a列为NA值的数据行&计算出各行是否含有“windows”
help(frame)
 |  copy : boolean, default False
 |      Copy data from inputs. Only affects DataFrame / 2d ndarray input
 '''
cframe=frame[frame.a.notnull()]#notnull函数返回true/false
operation_system=np.where(cframe.a.str.contains('Windows'),'Windows','Not Windows')#np : numpy module, cframe.a==cframe['a']
#group_by
by_tz_os=cframe.groupby(['tz',operation_system])
agg_counts=by_tz_os.size().unstack().fillna(0)
indexer=agg_counts.sum(1).argsort()#Ascending order，argsort函数返回的是数组值从小到大的索引值
count_subset=agg_counts.take(indexer)[-10:]#take the last 10 row in the order of indexer

#堆积（stacked=True）条形图（kind='barh'）
count_subset.plot(kind='barh',stacked='True')
#标准化（规范化为1）
normed_subset=count_subset.div(count_subset.sum(1),axis=0)
normed_subset.plot(kind='barh',stacked='True')


#movielen dataset
import pandas as pd
unames=['user_id','gender','age','occupation','zip']
users=pd.read_table(u'/Users/NoraChan/Desktop/python for data/ml-1m/users.dat',sep='::',header=None,names=unames)

rnames=['user_id','movie_id','rating','timestamp']
ratings=pd.read_table(u'/Users/NoraChan/Desktop/python for data/ml-1m/ratings.dat',sep='::',header=None,names=rnames)

mnames=['movie_id','title','genres']
movies=pd.read_table(u'/Users/NoraChan/Desktop/python for data/ml-1m/movies.dat',sep='::',header=None,names=mnames)


#利用merge函数讲ratings和users合并，再合并movies
data=pd.merge(pd.merge(ratings,users),movies)
#A primarily label-location based indexer, with integer position fallback.
data.ix[2:5]
#data.info(),get the introductino of Data

#pivot_table function to do the aggregation
mean_ratings=data.pivot_table('rating',index='title',columns='gender',aggfunc='mean') 
ratings_by_title=data.groupby('title').size()
#filter the movie with more than 250 rating records
active_titles=ratings_by_title.index[ratings_by_title>250]

#show the average rating of the movies above 
ratings_by_title.ix[active_titles]#size()
mean_ratings=mean_ratings.ix[active_titles]

#descend ratings by female
top_female_ratings=mean_ratings.sort_values(by='F',ascending=False)

#calculate the diff between 'M' and 'F'
mean_ratings['diff']=mean_ratings['M']-mean_ratings['F']
sorted_by_diff=mean_ratings.sort_values(by='diff')

#biggest bias
sorted_by_diff[::-1][:15]

#consider  standard deviation by title
rating_std_by_title=data.groupby('title')['rating'].std()

#filter by active_titles
rating_std_by_title=rating_std_by_title.ix[active_titles]

#descend by Series
rating_std_by_title.sort_values(ascending=False)[:10]#order sort_values sort_index  等价？

# cd to directory for  read_csv
import pandas as pd
names1880=pd.read_csv(u'/Users/NoraChan/Desktop/python for data/Baby Names from Social Security Card/yob1880.txt',names=['name','sex','births'])

#!head -n 10 yob1880.txt# read the head of yob1880.txt in UNIX system only works in the directory of the file
pd.read_csv(u'/Users/NoraChan/Desktop/python for data/Baby Names from Social Security Card/yob1880.txt',sep=',',names=['name','sex','births'])

'''
In [22]: pd.read_csv('yob1880.txt',sep=',',names=['name','sex','births']) is names1880
Out[22]: False
'''

names1880.groupby('sex').births.sum()#names1880.groupby('sex')['births'].sum()
#end by 2010
years=range(1880,2011)

#use a list to record seperate txt and add a column
pieces=[]
columns=['name','sex','births']
for year in years:
    path=u'/Users/NoraChan/Desktop/python for data/Baby Names from Social Security Card/yob%d.txt'%year
    frame=pd.read_csv(path,names=columns)
    frame['year']=year#add a column
    pieces.append(frame)

#use dataframe to represent the list
names=pd.concat(pieces,ignore_index=True)

#using groupby function
names.groupby(['sex','year']).size().unstack()
names.groupby(['sex','year']).births.sum().unstack()
total_births=names.pivot_table('births',index='year',columns='sex',aggfunc=sum)#aggfunc='sum'  is also OK

#what is it for?
total_births.tail()
#what is it for?
total_births.plot()

def add_group(group):
    births=group.births.astype(float)#avoid int. division
    group['prop']=births/births.sum()
    return group

#map reduce
names=names.groupby(['year','sex']).apply(add_group)

#
np.allclose(names.groupby(['year','sex']).prop.sum(),1)


def get_top1000(group):
    return group.sort_values(by='births',ascending=False)[:1000]

grouped=names.groupby(['year','sex'])
top1000=grouped.apply(get_top1000)

#get_top1000(names)  #what's the difference

#the way not using apply(map reduce fucntions)
#the ouput is weird
'''
pieces_2=[]
for year,group in names.groupby(['year','sex']):#names.groupby(['year','sex'] contains 2 value to unpack:(%year,%value),names.groupby(['year','sex']
    pieces_2.append(group.sort_values(by='births',ascending=False)[:1000])

top1000_2=pd.concat(pieces_2,ignore_index=True)
'''
#analytical operation
boys=top1000[top1000.sex=='M']
girls=top1000[top1000.sex=='F']

total_births=top1000.pivot_table('births',index='year',columns='name',aggfunc=sum)
subset=total_births[['John','Harry','Mary','Marilyn']]

subset.plot(subplots=True, figsize=(12,10),grid=False, title="Number of births per year")

table=top1000.pivot_table('prop',index='year',columns='sex',aggfunc=sum)#pay attention that it's top1000 dataset

table.plot(title='Sum of table1000.prop by year and sex',yticks=np.linspace(0,1.2,13),xticks=range(1880,2020,10))

df=boys[boys.year==2010]

prop_cumsum=df.sort_values(by='prop',ascending=False).prop.cumsum()
prop_cumsum[:10]

prop_cumsum.searchsorted(0.5)

def get_quantile_count(group,q=0.5):
    group=group.sort_index(by='prop',ascending=False)
    return (group.prop.cumsum().searchsorted(q)+1)[0]

diversity=top1000.groupby(['year','sex']).apply(get_quantile_count)
diversity=diversity.unstack('sex')

diversity.head()
diversity.plot(title="Number of popular names in top 50%")

get_last_letter= lambda x:x[-1]
last_letters=names.name.map(get_last_letter)
last_letters.name='last_letter'#column names

table=names.pivot_table('births',index=last_letters,columns=['sex','year'],aggfunc=sum)# how to match two seperate dataset?

subtable=table.reindex(columns=[1910,1960,2010],level='year')

subtable.sum()

letter_prop=subtable/subtable.sum().astype(float)

fig,axes=plt.subplots(2,1,figsize=(10,8))
letter_prop['M'].plot(kind='bar',rot=0,ax=axes[0],title='Male')
letter_prop['F'].plot(kind='bar',rot=0,ax=axes[1],title='Female',legend=False)

letter_prop=table/table.sum().astype(float)
dny_ts=letter_prop.ix[['d','n','y'],'M'].T
dny_ts.head()

dny_ts.plot()

all_names=top1000.name.unique()
mask=np.array(['lesl'in x.lower() for x in all_names])
lesley_like=all_names[mask]

lesley_like

filtered=top1000[top1000.name.isin(lesley_like)]
filtered.groupby('name').births.sum()

#aggregate by year and gender
table=filtered.pivot_table('births',index='year',columns='sex',aggfunc='sum')

table=table.div(table.sum(1),axis=0)

table.tail()

table.plot(style={'M':'k-','F':'k--'})


import numpy as np
data={i: randn() for i in range(7)}#data={i: randn() for i in range(7)}?

'''
#qt 
ipython qtconsole --pylab=inline
'''
ipython qtconsole
%pylab inline#insert pic in console

img=plt.imread(u'/Users/NoraChan/Desktop/python for data/ch3/stinkbug.png')
imshow(img)

ipython notebook
%pylab inline#insert pic in console


#chapter 4
data=randn(2,3)
data.shape
data.dtype

data1=[6,7.5,8,0,1]
arr1=np.array(data1)
data2=[[1,2,3,4],[5,6,7,8]]
arr2=np.array(data2)#data2的多维数组列数需要一致
arr2.ndim
arr2.dtype


'''
data3=[[1,2],[1,2,3]]#列数不一致-->dtype==object
np.array(data3)
np.array(data3,dtype=np.int32)#setting an array element with a sequence.
'''
#initiate with 0
np.zeros(10)
np.zeros((2,3))
#initiate with some data
np.empty((2,3))

np.arange(15)


arr2.astype(float)#automatically converting arr1.astype(np.float64)
arr1.astype(int)#

np.array(['1.25','-9.6','42'],dtype=np.string_).astype(np.float64)

arr=np.arange(10)
arr[5:8]=12
arr_slice=arr[5:8]#arr_slice=arr[5:8].copy()
arr_slice[1]=12345

arr2d=np.array([[1,2,3],[4,5,6],[7,8,9]])
arr2d[1,2]#arr2d[1][2]

arr3d=np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
old_values=arr3d[0].copy()
arr3d[0]=42#each elements in the array are changed
arr3d
arr3d[0]=old_values

#slice
arr2d
arr2d[:2]
arr2d[:2,1:]
arr2d[:,:1]
arr2d[:2,1:]=0

#boolean index
names=np.array(['Bob','Joe','Will','Bob','Will','Joe','Joe'])
data=randn(7,4)
#return boolean array
names=='Bob'
#use the return to be the index
names[names=='Bob']
data[names=='Bob']

data[names!='Bob']
data[-(names=='Bob')]
data[(names=='Bob')|(names=='Will')]# the keyword of Python,'and'/'or',doesn't work in boolean type


#conbined with list
data[names=='Bob',2:]


#fancy indexing
arr=np.empty((8,4))
for i in range(8):
    arr[i]=i

arr[[4,3,0,6]]
arr[[-3,-5,-7]]

#reshape
arr=np.arange(32).reshape((8,4))
arr[[1,5,7,2],[0,3,1,2]]#return (1,0)(5,3)(7,1)(2,2) elements of the array

arr[[1,5,7,2]][:,[0,3,1,2]]#martix area
arr[np.ix_([1,5,7,2],[0,3,1,2])]

#transpose
arr=np.arange(15).reshape((3,5))
arr.T
#
arr=np.random.randn(6,3)
np.dot(arr.T,arr)

#multi-dimension
arr=np.arange(16).reshape((4,4))
arr=arr.reshape((2,2,4))
arr.T==arr.transpose((2,1,0))#true
arr.transpose((1,0,2))
'''
arr1.shape 应该是(2, 2, 4) 意为 2维，2*4矩阵

arr1.transpose(*args) 里面的参数，可以这么理解，他是调换arr1.shape的顺序，咱来给arr1.shape标一下角标哈，（2[0], 2[1], 4[2]）  [ ] 里是shape的索引，对吧， 
transpose((1, 0, 2)) 的意思是 按照这个顺序 重新设置shape 也就是 （2[1], 2[0], 4[2]）

虽然看起来 变换前后的shape都是 2,2,4  ， 但是问题来了，transpose是转置
shape按照(1,0,2)的顺序重新设置了， array里的所有元素 也要按照这个规则重新组成新矩阵

比如 8 在arr1中的索引是 (1, 0, 0)  那么按照刚才的变换规则，就是 (0, 1, 0) 看看跟你结果arr2的位置一样了吧，依此类推...
'''
arr.swapaxes(1,2)

arr=np.arange(10)
np.sqrt(arr)
np.exp(arr)
np.modf(arr)

points=np.arange(-5,5,0.01)
xs,ys=np.meshgrid(points,points)#make a mesh

# vectored calculation
import matplotlib.pyplot as plt
z=np.sqrt(xs**2+ys**2)
plt.imshow(z,cmap=plt.cm.gray);plt.colorbar()
plt.title("Image plot of $\sqrt{x^2+y^2}$ fro a grid of values")

#use arrray as condition
xarr=np.array([1.1,1.2,1.3,1.4,1.5])
yarr=np.array([2.1,2.2,2.3,2.4,2.5])
cond=np.array([True,False,True,True,False])

result=[(x if c else y)
            for x,y,c in zip(xarr,yarr,cond)]
#equals
result=np.where(cond,xarr,yarr)

#an example for np.where
arr=randn(4,4)
np.where(arr>0,2,-2)
#or, not equal
np.where(arr>0,2,arr)

#stastics
arr=np.random.randn(5,4)
arr.mean()
np.mean(arr)
arr.sum()
arr.mean(axis=1)
arr.sum(0)#arr.sum(axis=0)

arr=np.arange(9).reshape(3,3)#arr=np.array([[0,1,2],[3,4,5],[6,7,8]])
arr.cumsum(0)#by column
arr.cumprod(1)#by row

#true==1 ,calculate the number of condition
arr=randn(100)
(arr>0).sum()#count() of postive num
bools=np.array([False,False,True,False])
bools.any()
bools.all()

arr=randn(8)
arr.sort()
arr=randn(5,3)
arr.sort(1)#arr.sort(0)
#5%
large_arr=randn(1000)
large_arr.sort()
large_arr[int(0.05*len(large_arr))]


names=np.array(['Bob','Joe','Will','Bob','Will','Joe','Joe'])
np.unique(names)#sorted
#pure python sorted
sorted(set(names))

values=np.array([6,0,0,3,2,5,6])
np.in1d(values,[2,3,6])

x=np.array([[1.,2.,3.],[4.,5.,6.]])
y=np.array([[6.,23],[-1,7],[8.,9]])

#dot product
np.dot(x,np.ones(3))# np.ones(3).ndim=1
'''
np.dot(np.ones(3),x)#illegal
'''
#decomposition
from numpy.linalg import inv,qr
X=randn(5,5)
mat=X.T.dot(X)
inv(mat)
mat.dot(inv(mat))

q,r=qr(mat)

#normal
samples=np.random.normal(size=(4,4))
samples

from random import normalvariate
N=1000000
%timeit samples=[normalvariate(0,1) for _ in xrange(N)]
%timeit np.random.normal(size=N)

#chapter 5
from pandas import Series,DataFrame
import pandas as pd
obj=Series([4,7,-5,3])
obj.index
obj.values

obj2=Series([4,7,-5,3],index=['d','b','a','c'])

obj2[['c','a','d']]
obj2>0

obj2[obj2>0]
obj2*2
np.exp(obj2)

'b' in obj2
#Series by dict
sdata={'Ohio':35000,'Texas':71000,'Oregon':16000,'Utah':5000}
obj3=Series(sdata)

states=['California','Ohio','Oregon','Texas']
obj4=Series(sdata,index=states)

pd.isnull(obj4)
pd.notnull(obj4)

obj4.isnull()
obj4.name='population'
obj4.index.name='state'

obj.index=['Bob','Steve','Jeff','Ryan']#modify

#dict construct DataFrame
data={'state':['Ohio','Ohio','Ohio','Nevada','Nevada'],
        'year':[2000,2001,2002,2001,2002],
        'pop':[1.5,1.7,3.6,2.4,2.9]}

frame=DataFrame(data)#fill with NA
frame=DataFrame(data,columns=['year','state','pop'])
frame2=DataFrame(data,columns=['year','state','pop'],index=['one','two','three','four','five'])

frame2['state']
frame2.year

frame2.ix['three']
frame2.year.ix['three']
frame2['debt']=16.5
frame2['debt']=np.arange(5)

val=Series([-1.2,-1.5,-1.7],index=['two','four','five'])
frame2['debt']=val
frame2['eastern']=frame2.state=='Ohio'
del frame2['eastern']


pop={'Nevada':{2001:2.4,2002:2.9},'Ohio':{2000:1.5,2001:1.7,2002:3.6}}
frame3=DataFrame(pop)
frame3.T

DataFrame(pop,index=[2001,2002,2003])

pdata={'Ohio':frame3['Ohio'][:-1],'Nevada':frame3['Nevada'][:2]}

DataFrame(pdata)

frame3.index.name='year'
frame3.columns.name='state'
frame3.values
frame2.values

obj=Series(range(3),index=['a','b','c'])
index=obj.index
index[1:]

index[1]='d'#index cannot be modified

index=pd.Index(np.arange(3))

obj2=Series([1.5,2.5,0],index=index)
obj2.index is index
obj2.index=pd.Index(np.arange(4,7))#why it can be mutual

'Ohio' in frame3.columns
2003 in frame3.index

#how to reindex
obj=Series([4.5,7.2,-5.3,3.6],index=['d','b','a','c'])
obj2=obj.reindex(['a','b','c','d','e'])
obj2=obj.reindex(['a','b','c','d','e'],fill_value=0)

obj3=Series(['blue','purple','yellow'],index=[0,2,4])
obj3.reindex(range(6),method='ffill')


frame=DataFrame(np.arange(9).reshape((3,3)),index=['a','c','d'],columns=['Ohio','Texas','California'])

frame2=frame.reindex(['a','b','c','d'])
states=['Texas','Utah','California']
frame.reindex(columns=states)

frame.reindex(index=['a','b','c','d'],method='ffill',columns=states)

frame.ix[['a','b','c','d'],states]#reindex by NAN

obj=Series(np.arange(5.),index=['a','b','c','d','e'])
new_obj=obj.drop('c')
obj.drop(['d','c'])#return a new object
obj

data=DataFrame(np.arange(16).reshape((4,4)),index=['Ohio','Colorado','Utah','New York'],columns=['one','two','three','four'])


data.drop(['Colorado','Ohio'])
data.drop('two',axis=1)
data.drop(['two','four'],axis=1)#axis : row/column


obj=Series(np.arange(4.),index=['a','b','c','d'])
obj['b']
obj[1]
obj[2:4]
obj[['b','a','d']]
obj[[1,3]]
obj[obj<2]
obj['b':'d']
obj['b':'d']=5


data=DataFrame(np.arange(16).reshape((4,4)),index=['Ohio','Colorado','Utah','New York'],columns=['one','two','three','four'])

data['two']#index for column
data[['three','one']]
data[0:2]
data[data['three']>5]#data['three']>5   returns   [False,  True,  True,  True]

data<5
data[data<5]=0

data.ix['Colorado',['two','three']]
data.ix[2]
data.ix[data.three>5,:3]

s1=Series([7.3,-2.5,3.4,1.5],index=['a','c','d','e'])
s2=Series([-2.1,3.6,-1.5,4,3.1],index=['a','c','e','f','g'])
s1+s2

df1=DataFrame(np.arange(9.).reshape((3,3)),columns=list('bcd'),index=['Ohio','Texas','Colorado'])

df2=DataFrame(np.arange(12.).reshape((4,3)),columns=list('bde'),index=['Utah','Ohio','Texas','Oregon'])

df1+df2

df1=DataFrame(np.arange(12.).reshape((3,4)),columns=list('abcd'))

df2=DataFrame(np.arange(20.).reshape((4,5)),columns=list('abcde'))

df1+df2

df1.add(df2,fill_value=0)

df1.reindex(columns=df2.columns,fill_value=0)


#Caculation between Series and DataFrame
#Numpy type
arr=np.arange(12.).reshape((3,4))
arr[0]
arr-arr[0]

#DataFrame & Series
frame=DataFrame(np.arange(12.).reshape((4,3)),columns=list('bde'),index=['Utah','Ohio','Texas','Oregon'])
series=frame.ix[0]
frame-series
series2=Series(range(3),index=['b','e','f'])
series2+frame
series3=frame['d']
frame.sub(series3,axis=0)


frame=DataFrame(np.random.randn(4,3),columns=list('bde'),index=['Utah','Ohio','Texas','Oregon'])
np.abs(frame)
f=lambda x:x.max()-x.min()
frame.apply(f)
frame.apply(f,axis=1)

def f(x):
    return Series([x.min(),x.max()],index=['min','max'])

frame.apply(f)
frame.apply(f,axis=1)

format=lambda x:'%.2f' %x
frame.applymap(format)
frame['e'].map(format)

obj=Series(range(4),index=['d','a','b','c'])
obj.sort_index()

frame=DataFrame(np.arange(8).reshape((2,4)),index=['three','one'],columns=['d','a','b','c'])
frame.sort_index()
frame.sort_index(axis=1)
frame.sort_index(axis=1,ascending=False)

obj=Series([4,7,-3,2])
obj.sort_values()
obj=Series([4,np.nan,7,np.nan,-3,2])
obj.sort_values()


frame=DataFrame({'b':[4,7,-3,2],'a':[0,1,0,1]})
frame.sort_values(by='b')
frame.sort_values(by=['a','b'])

obj=Series([7,-5,7,4,2,0,4])

obj.rank()

obj.rank(method='first')
obj.rank(ascending=False,method='max')
frame=DataFrame({'b':[4.3,7,-3,2],'a':[0,1,0,1],'c':[-2,5,8,-2.5]})
frame.rank(axis=1)

obj=Series(range(5),index=['a','a','b','b','c'])
obj.index.is_unique

obj['a']
obj['c']
df=DataFrame(np.random.randn(4,3),index=['a','a','b','b'])
df.ix['b']

df=DataFrame([[1.4,np.nan],[7.1,4.5],[np.nan,np.nan],[0.75,-1.3]],index=['a','b','c','d'],columns=['one','two'])
df.sum()
df.sum(axis=1)
df.sum(axis=1,skipna=False)

df.idmxax()
df.cumsum()
df.describe()

obj=Series(['a','a','b','c'*4])
obj.describe()

import pandas_datareader as web#import pandas.io.data as web
all_data={}
for ticker in ['AAPL','IBM','MSFT','GOOG']:
    all_data[ticker]=web.get_data_yahoo(ticker,'1/1/2000','1/1/2010')

price=DataFrame({tic:data['Adj Close'] for tic,data in all_data.iteritems()})
volume=DataFrame({tic:data['Volume'] for tic,data in all_data.iteritems()})

returns=price.pct_change()
returns.tail()

returns.MSFT.corr(returns.IBM)
returns.MSFT.cov(returns.IBM)

returns.corr()
returns.cov()

returns.corrwith(returns.IBM)
returns.corrwith(volume)

#chapter 6
!cat /Users/NoraChan/Desktop/python for data/ch06/ex1.csv

import pandas as pd
df=pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex1.csv')

!cat /Users/NoraChan/Desktop/python for data/ch06/ex2.csv
pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex2.csv',header=None)

pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex2.csv',names=['a','b','c','d','message'])

names=['a','b','c','d','message']
pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex2.csv',names=names,index_col='message')

!cat /Users/NoraChan/Desktop/python for data/ch06/csv_mindex.csv
parsed=pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/csv_mindex.csv',index_col=['key1','key2'])

list(open(u'/Users/NoraChan/Desktop/python for data/ch06/ex3.txt'))

result=pd.read_table(u'/Users/NoraChan/Desktop/python for data/ch06/ex3.txt',sep='\s+')

!cat /Users/NoraChan/Desktop/python for data/ch06/ex4.csv
pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex4.csv',skiprows=[0,2,3])

!cat /Users/NoraChan/Desktop/python for data/ch06/ex5.csv
result=pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex5.csv')

pd.isnull(result)
result=pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex5.csv',na_values=['NULL'])

sentinels={'message':['foo','NA'],'something':['two']}
pd.read_csv(u'/Users/NoraChan/Desktop/python for data/ch06/ex5.csv',na_values=sentinels)

obj="""
{"name":"Wes",
"places_lived":["United States","Spain","Germany"],
"pet":null,
"siblings":[{"name":"Scott","age":"25","pet":"Zuko"},
{"name":"Katie","age":33,"pet":"Cisco"}]
}
"""

import json
result=json.loads(obj)

asjson=json.dumps(result)

#DataFrame for Json
#to_json
#from_json
siblings=DataFrame(result['siblings'],columns=['name','age'])

from lxml.html import parse
from urllib2 import urlopen

parsed=parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc=parsed.getroot()

links=doc.findall('.//a')
links[15:20]

lnk=links[28]
lnk.get('href')
lnk.text_content()

urls=[lnk.get('href') for lnk in doc.findall('.//a')]

urls[-10:]

tables=doc.findall('.//table')
calls=tables[9]
puts=tables[13]

rows=calls.findall('.//tr')

def _unpack(row,kind='td'):
    elts=row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

import requests
url='https://api.twitter.com/1.1/search/tweets.json?q=python%20pandas'
resp=requests.get(url)
resp

import json
data=json.loads(resp.text)
data.keys()

import sqlite3
query="""
CREATE TABLE test
(a VARCHAR(20),b VARCHAR(20),
c REAL, d INTEGER
);"""
con=sqlite3.connect(':memory:')
con.execute(query)
con.commit()


data=[('Atlanta','Georgia',1.25,6),('Tallahassee','Florida',2.6,3),('Sacramento','California',1.7,5)]
stmt='INSERT INTO test VALUES(?,?,?,?)'


con.executemany(stmt,data)
con.commit()

cursor=con.execute('select * from test')
rows=cursor.fetchall()

cursor.description

DataFrame(rows,columns=zip(*cursor.description)[0])


import pandas.io.sql as sql
sql.read_sql('select * from test',con)#read_frame

import pymongo
#con=pymongo.Connection('localhost',port=27017)



#chapter 7
ipython --pylab
plot(np.arange(10))

import matplotlib.pyplot as plt

fig=plt.figure()

ax1=fig.add_subplot(2,2,1)
ax2=fig.add_subplot(2,2,2)
ax3=fig.add_subplot(2,2,3)


plt.plot([1.5,3.5,-2,1.6])

from numpy.random import randn
plt.plot(randn(50).cumsum(),'k--')

_=ax1.hist(randn(100),bins=20,color='k',alpha=0.3)
ax2.scatter(np.arange(30),np.arange(30)+3*randn(30))

fig,axes=plt.subplots(2,3)
axes

axes[0,1]

subplots_adjust(left=None,bottom=None,right=None,top=None,wspace=None,hspace=None)

fig,axes=plt.subplots(2,2,sharex=True,sharey=True)
for i in range(2):
    for j in range(2):
        axes[i,j].hist(randn(500),bins=50,color='k',alpha=0.5)

plt.subplots_adjust(wspace=0,hspace=0)

#sample
'''
ax.plot(x,y,'g--')
ax.plot(x,y,linestyle='--',color='g')
'''


plt.plot(randn(30).cumsum(),'ko--')
plt.plot(randn(30).cumsum(),color='k',linestyle='dashed',marker='o')

#modify by 'drawstyle' options
data=randn(30).cumsum()
plt.plot(data,'k--',label='Default')
plt.plot(data,'k-',drawstyle='steps-post',label='steps-post')

plt.legend(loc='best')

#xlim -> x axis limitation
#xticks
#xticklabels

fig=plt.figure();
ax=fig.add_subplot(1,1,1)
ax.plot(randn(1000).cumsum())

#set_xticks;set_xticklabels
ticks=ax.set_xticks([0,250,500,750,1000])
label=ax.set_xticklabels(['one','two','three','four','five'],rotation=30,fontsize='small')

#set_xlabel; set_title
ax.set_title('My first matplotlib plot')
ax.set_xlabel('Stages')

#y axis has a similar way to set

#legend

fig=plt.figure();ax=fig.add_subplot(1,1,1)
ax.plot(randn(1000).cumsum(),'k',label='one')
ax.plot(randn(1000).cumsum(),'k--',label='two')
ax.plot(randn(1000).cumsum(),'k.',label='three')

ax.legend(loc='best')

#label='_nolegend_' or do not pass 'label'


#chapter 8
from pandas import DataFrame,Series
import pandas as pd;import numpy as np
df=DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})

grouped=df['data1'].groupby(df['key1'])
grouped#have not caculate but support all relative groupby calculation

grouped.mean()

means=df['data1'].groupby([df['key1'],df['key2']]).mean()

means.unstack()

#group by array type
states=np.array(['Ohio','California','California','Ohio','Ohio'])
years=np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

df.groupby('key1').mean()

df.groupby('key1','key2').mean()

#return groupby size
df.groupby(['key1','key2']).size()

#iteration
for name,group in df.groupby('key1'):
    print name#key1
    print group#df1 by key1

for (k1,k2),group in df.groupby(['key1','key2']):
    print k1,k2
    print group

#Make a dict
pieces=dict(list(df.groupby('key1')))
pieces['b']

#group by dtype,change the axis
grouped=df.groupby(df.dtypes,axis=1)
dict(list(grouped))

#filter
df.groupby('key1')['data1']#df['data1'].groupby(df['key1'])
df.groupby('key1')['data2']#df['data2'].groupby(df['key1'])

df.groupby(['key1','key2'])[['data2']].mean()

#group by dict
people=DataFrame(np.random.randn(5,5),columns=['a','b','c','d','e'],index=['Joe','Steve','Wes','Jim','Travis'])
people.ix[2:3,['b','c']]=np.nan# add NA

mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

by_column=people.groupby(mapping,axis=1)
by_column.sum()

#group by Series
map_series=Series(mapping)
people.groupby(map_series,axis=1).count()

#group by function
people.groupby(len).sum()

#group by mix
key_list=['one','one','one','two','two']
people.groupby([len,key_list]).min()

#group by level
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])
hier_df=DataFrame(np.random.randn(4,5),columns=columns)
hier_df.groupby(level='cty',axis=1).count()

RuntimeError: Invalid DISPLAY variable